In [1]:
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Dense, Dropout
from sklearn.metrics import roc_auc_score, classification_report, confusion_matrix
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
def count_null(df):
    null_counts_per_column = []

    for column in df.columns:
        null_count = df[column].isnull().sum()
        null_counts_per_column.append((column, null_count))

    return null_counts_per_column

def count_zero(df, column):
    print(df[df[column] == 0].shape, df[df[column] != 0].shape)


In [77]:
import pandas as pd
from tabulate import tabulate


# Create a DataFrame with the initial specified data
data = {
    'Method': ['initial'], 'n_neg': [1492937], 'n_pos': [1937],
    'n_neg:n_pos': ['999:1'], 'Decision Threshold': [0.001],
    'AUC': [0.8261], 'TPR': [0.8552], 'TNR': [0.5834], 'G-Mean' : [0.7064]
}

data_1 = {
    'Method': ['RUS_1'], 'n_neg': [191763], 'n_pos': [1937],
    'n_neg:n_pos': ['99:1'], 'Decision Threshold': [0.01],
    'AUC': [0.8628], 'TPR': [0.7803], 'TNR': [0.7979], 'G-Mean' : [0.7891]
}

# Additional metrics to be added
data_2 = {
    'Method': ['RUS_2'], 'n_neg': [7748], 'n_pos': [1937],  
    'n_neg:n_pos': ['4:1'], 'Decision Threshold': [0.2],  
    'AUC': [0.8315],  'TPR': [0.6336], 'TNR': [0.8433],'G-Mean' : [0.7309]
}

data_3 = {
    'Method': ['RUS_3'], 'n_neg': [2905], 'n_pos': [1937],  
    'n_neg:n_pos': ['3:2'], 'Decision Threshold': [0.4],  
    'AUC': [0.8651], 'TPR': [0.7835], 'TNR': [0.7486], 'G-Mean' : [0.7986]
}
data_4 = {
    'Method': ['RUS_4'], 'n_neg': [1937], 'n_pos': [1937],  
    'n_neg:n_pos': ['1:1'], 'Decision Threshold': [0.5],  
    'AUC': [0.8715], 'TPR': [0.8372], 'TNR': [0.7598], 'G-Mean' : [0.7912]
}
data_5 = {
    'Method': ['RUS_5'], 'n_neg': [1937], 'n_pos': [1291],  
    'n_neg:n_pos': ['2:3'], 'Decision Threshold': [0.573],  
    'AUC': [0.8527], 'TPR': [0.8067], 'TNR': [0.7596], 'G-Mean' : [0.7828]
}

df = pd.DataFrame(data)
# Update DataFrame with new data
df = pd.concat([df, pd.DataFrame(data_1), pd.DataFrame(data_2), pd.DataFrame(data_3), pd.DataFrame(data_4),pd.DataFrame(data_5)]).reset_index(drop=True)

# Format numbers with commas for thousands and percentages/decimals appropriately
for col in ['AUC', 'TPR', 'TNR']:
    df[col] = df[col].apply(lambda x: f"{x:.4f}")

df['n_neg'] = df['n_neg'].apply(lambda x: f"{x:,}")
df['n_pos'] = df['n_pos'].apply(lambda x: f"{x:,}")

# Print the DataFrame using tabulate
print(tabulate(df, headers='keys', tablefmt='grid', showindex=False))


+----------+-----------+---------+---------------+----------------------+--------+--------+--------+----------+
| Method   | n_neg     | n_pos   | n_neg:n_pos   |   Decision Threshold |    AUC |    TPR |    TNR |   G-Mean |
+==========+===========+=========+===============+======================+========+========+========+==========+
| initial  | 1,492,937 | 1,937   | 999:1         |                0.001 | 0.8261 | 0.8552 | 0.5834 |   0.7064 |
+----------+-----------+---------+---------------+----------------------+--------+--------+--------+----------+
| RUS_1    | 191,763   | 1,937   | 99:1          |                0.01  | 0.8628 | 0.7803 | 0.7979 |   0.7891 |
+----------+-----------+---------+---------------+----------------------+--------+--------+--------+----------+
| RUS_2    | 7,748     | 1,937   | 4:1           |                0.2   | 0.8315 | 0.6336 | 0.8433 |   0.7309 |
+----------+-----------+---------+---------------+----------------------+--------+--------+--------+----

In [2]:
df_final = pd.read_csv("data_set_result_aggregated_by_npi.csv")

In [49]:
df_final.shape
df_final[df_final['fraud'] == 1].shape
# df_final[df_final['fraud'] == 0].shape
df_final.columns

majority = df_final[df_final['fraud'] == 0]
minority = df_final[df_final['fraud'] == 1]

has_nan = df_final.isna().any().any()
print("Are there any NaN values in df_final?", has_nan)


# Replace 'column_name' with your actual column name
count_null(df_final)
df_final.fillna(0, inplace=True)

(1494874, 153)

(1937, 153)

Index(['rndrng_npi', 'fraud', 'tot_benes_mean', 'tot_benes_sum',
       'tot_benes_median', 'tot_benes_std', 'tot_benes_min', 'tot_benes_max',
       'tot_srvcs_mean', 'tot_srvcs_sum',
       ...
       'rndrng_prvdr_type_advanced heart failure and transplant cardiology',
       'rndrng_prvdr_type_clinical cardiac electrophysiology',
       'rndrng_prvdr_type_medical toxicology',
       'rndrng_prvdr_type_oral surgery (dentist only)',
       'rndrng_prvdr_type_hematopoietic cell transplantation and cellular therapy',
       'rndrng_prvdr_type_medical genetics and genomics',
       'rndrng_prvdr_type_pharmacy',
       'rndrng_prvdr_type_undersea and hyperbaric medicine',
       'rndrng_prvdr_type_adult congenital heart disease',
       'rndrng_prvdr_type_micrographic dermatologic surgery'],
      dtype='object', length=153)

Are there any NaN values in df_final? False


[('rndrng_npi', 0),
 ('fraud', 0),
 ('tot_benes_mean', 0),
 ('tot_benes_sum', 0),
 ('tot_benes_median', 0),
 ('tot_benes_std', 0),
 ('tot_benes_min', 0),
 ('tot_benes_max', 0),
 ('tot_srvcs_mean', 0),
 ('tot_srvcs_sum', 0),
 ('tot_srvcs_median', 0),
 ('tot_srvcs_std', 0),
 ('tot_srvcs_min', 0),
 ('tot_srvcs_max', 0),
 ('tot_bene_day_srvcs_mean', 0),
 ('tot_bene_day_srvcs_sum', 0),
 ('tot_bene_day_srvcs_median', 0),
 ('tot_bene_day_srvcs_std', 0),
 ('tot_bene_day_srvcs_min', 0),
 ('tot_bene_day_srvcs_max', 0),
 ('avg_sbmtd_chrg_mean', 0),
 ('avg_sbmtd_chrg_sum', 0),
 ('avg_sbmtd_chrg_median', 0),
 ('avg_sbmtd_chrg_std', 0),
 ('avg_sbmtd_chrg_min', 0),
 ('avg_sbmtd_chrg_max', 0),
 ('avg_mdcr_pymt_amt_mean', 0),
 ('avg_mdcr_pymt_amt_sum', 0),
 ('avg_mdcr_pymt_amt_median', 0),
 ('avg_mdcr_pymt_amt_std', 0),
 ('avg_mdcr_pymt_amt_min', 0),
 ('avg_mdcr_pymt_amt_max', 0),
 ('rndrng_prvdr_type_addiction medicine', 0),
 ('rndrng_prvdr_type_all other suppliers', 0),
 ('rndrng_prvdr_type_allergy/i

In [50]:
df_final.shape
df_final.columns
df_final[df_final['fraud']==1]
sum(df_final['rndrng_prvdr_type_medical toxicology'])

(1494874, 153)

Index(['rndrng_npi', 'fraud', 'tot_benes_mean', 'tot_benes_sum',
       'tot_benes_median', 'tot_benes_std', 'tot_benes_min', 'tot_benes_max',
       'tot_srvcs_mean', 'tot_srvcs_sum',
       ...
       'rndrng_prvdr_type_advanced heart failure and transplant cardiology',
       'rndrng_prvdr_type_clinical cardiac electrophysiology',
       'rndrng_prvdr_type_medical toxicology',
       'rndrng_prvdr_type_oral surgery (dentist only)',
       'rndrng_prvdr_type_hematopoietic cell transplantation and cellular therapy',
       'rndrng_prvdr_type_medical genetics and genomics',
       'rndrng_prvdr_type_pharmacy',
       'rndrng_prvdr_type_undersea and hyperbaric medicine',
       'rndrng_prvdr_type_adult congenital heart disease',
       'rndrng_prvdr_type_micrographic dermatologic surgery'],
      dtype='object', length=153)

,rndrng_npi,fraud,tot_benes_mean,tot_benes_sum,tot_benes_median,tot_benes_std,tot_benes_min,tot_benes_max,tot_srvcs_mean,tot_srvcs_sum,...,rndrng_prvdr_type_advanced heart failure and transplant cardiology,rndrng_prvdr_type_clinical cardiac electrophysiology,rndrng_prvdr_type_medical toxicology,rndrng_prvdr_type_oral surgery (dentist only),rndrng_prvdr_type_hematopoietic cell transplantation and cellular therapy,rndrng_prvdr_type_medical genetics and genomics,rndrng_prvdr_type_pharmacy,rndrng_prvdr_type_undersea and hyperbaric medicine,rndrng_prvdr_type_adult congenital heart disease,rndrng_prvdr_type_micrographic dermatologic surgery
3,1003000407,1,0.005565,0.008145,0.003619,0.002456,0.000000,0.003382,0.000752,0.003203,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
86,1003005315,1,0.002044,0.002229,0.000845,0.001198,0.000000,0.001667,0.000270,0.000770,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
821,1003042441,1,0.009282,0.000357,0.009092,0.001353,0.003744,0.001713,0.000988,0.000166,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1348,1003066838,1,0.003326,0.002740,0.002459,0.000700,0.000000,0.001188,0.000314,0.001156,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1987,1003095381,1,0.007679,0.004491,0.002779,0.001994,0.000000,0.005043,0.001454,0.002019,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1490924,1992814990,1,0.001755,0.001008,0.000820,0.001949,0.000000,0.001751,0.000601,0.000912,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1491606,1992851836,1,0.002378,0.000376,0.002767,0.000242,0.000651,0.000508,0.002390,0.000882,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1491835,1992860977,1,0.003264,0.002022,0.003256,0.002458,0.000111,0.002069,0.000670,0.000951,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1492149,1992873772,1,0.000965,0.000774,0.000683,0.001194,0.000000,0.001111,0.000280,0.000438,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


49.0

In [52]:
X = df_final.drop(['fraud', 'rndrng_npi'], axis=1)
y = df_final['fraud']

# Splitting data into training and testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Normalizing the data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Building the neural network model
model = Sequential([
    Dense(32, input_dim=X_train_scaled.shape[1], activation='relu'),
    Dropout(0.5),
    Dense(32, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(X_train_scaled, y_train, epochs=50, batch_size=32, validation_split=0.1)

# Predict probabilities for the test set
predictions = model.predict(X_test_scaled)

# Evaluate the model
print("ROC AUC Score:", roc_auc_score(y_test, predictions.flatten()))

# Adjust threshold and compute binary outcomes
threshold = 0.001
predicted_classes = (predictions > threshold).astype(int)

# Calculate and display the confusion matrix
cm = confusion_matrix(y_test, predicted_classes)
TN, FP, FN, TP = cm.ravel()

# Calculate TPR and TNR
TPR = TP / (TP + FN)  # True Positive Rate
TNR = TN / (TN + FP)  # True Negative Rate

# Calculate the G-Mean
G_mean = np.sqrt(TPR * TNR)

# Display TPR, TNR, and G-Mean
print("True Positive Rate (TPR):", TPR)
print("True Negative Rate (TNR):", TNR)
print("Geometric Mean (G-Mean):", G_mean)

# Detailed classification report
print(classification_report(y_test, predicted_classes))

C:\Users\temir\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:86: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
33635/33635 ━━━━━━━━━━━━━━━━━━━━ 44s 1ms/step - accuracy: 0.9898 - loss: 0.0364 - val_accuracy: 0.9987 - val_loss: 0.0088
Epoch 2/50
33635/33635 ━━━━━━━━━━━━━━━━━━━━ 84s 1ms/step - accuracy: 0.9986 - loss: 0.0103 - val_accuracy: 0.9987 - val_loss: 0.0087
Epoch 3/50
33635/33635 ━━━━━━━━━━━━━━━━━━━━ 80s 1ms/step - accuracy: 0.9987 - loss: 0.0097 - val_accuracy: 0.9987 - val_loss: 0.0091
Epoch 4/50
33635/33635 ━━━━━━━━━━━━━━━━━━━━ 41s 1ms/step - accuracy: 0.9986 - loss: 0.0102 - val_accuracy: 0.9987 - val_loss: 0.0090
Epoch 5/50
33635/33635 ━━━━━━━━━━━━━━━━━━━━ 42s 1ms/step - accuracy: 0.9987 - loss: 0.0099 - val_accuracy: 0.9987 - val_loss: 0.0092
Epoch 6/50
33635/33635 ━━━━━━━━━━━━━━━━━━━━ 80s 1ms/step - accuracy: 0.9987 - loss: 0.0102 - val_accuracy: 0.9987 - val_loss: 0.0089
Epoch 7/50
33635/33635 ━━━━━━━━━━━━━━━━━━━━ 42s 1ms/step - accuracy: 0.9987 - loss: 0.0098 - val_accuracy: 0.9987 - val_loss: 0.0091
Epoch 8/50
33635/33635 ━━━━━━━━━━━━━━━━━━━━ 39s 1ms/step - accuracy: 

9343/9343 ━━━━━━━━━━━━━━━━━━━━ 7s 693us/step
ROC AUC Score: 0.8261562976045497
True Positive Rate (TPR): 0.8552971576227391
True Negative Rate (TNR): 0.5834728790172411
Geometric Mean (G-Mean): 0.7064295399920665
              precision    recall  f1-score   support

           0       1.00      0.58      0.74    298588
           1       0.00      0.86      0.01       387

    accuracy                           0.58    298975
   macro avg       0.50      0.72      0.37    298975
weighted avg       1.00      0.58      0.74    298975



In [53]:
threshold = 0.00126
predicted_classes = (predictions > threshold).astype(int)

# Calculate and display the confusion matrix
cm = confusion_matrix(y_test, predicted_classes)
TN, FP, FN, TP = cm.ravel()

# Calculate TPR and TNR
TPR = TP / (TP + FN)  # True Positive Rate
TNR = TN / (TN + FP)  # True Negative Rate

# Calculate the G-Mean
G_mean = np.sqrt(TPR * TNR)

# Display TPR, TNR, and G-Mean
print("True Positive Rate (TPR):", TPR)
print("True Negative Rate (TNR):", TNR)
print("Geometric Mean (G-Mean):", G_mean)

# Detailed classification report
print(classification_report(y_test, predicted_classes))

True Positive Rate (TPR): 0.7803617571059431
True Negative Rate (TNR): 0.7472972791940734
Geometric Mean (G-Mean): 0.7636505862450298
              precision    recall  f1-score   support

           0       1.00      0.75      0.86    298588
           1       0.00      0.78      0.01       387

    accuracy                           0.75    298975
   macro avg       0.50      0.76      0.43    298975
weighted avg       1.00      0.75      0.85    298975



In [54]:
#RUS to majority class to be equal to 99 * minority class
ratio = 99
# Here, we match the number of instances in the minority class
majority_undersampled = majority.sample(n=ratio * len(minority), random_state=42)  # `random_state` ensures reproducibility
# Concatenate the minority class with the undersampled majority class
df_RUS_1 = pd.concat([majority_undersampled, minority])
# Shuffle the dataframe to prevent the model from learning any order
df_RUS_1 = df_RUS_1.sample(frac=1, random_state=42).reset_index(drop=True)
# Verify the number of instances in each class
print(df_RUS_1['fraud'].value_counts())
df_RUS_1.shape

fraud
0    191763
1      1937
Name: count, dtype: int64


(193700, 153)

In [55]:
X = df_RUS_1.drop(['fraud', 'rndrng_npi'], axis=1)
y = df_RUS_1['fraud']

# Splitting data into training and testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Normalizing the data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Building the neural network model
model = Sequential([
    Dense(32, input_dim=X_train_scaled.shape[1], activation='relu'),
    Dropout(0.5),
    Dense(32, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(X_train_scaled, y_train, epochs=50, batch_size=32, validation_split=0.1)

# Predict probabilities for the test set
predictions = model.predict(X_test_scaled)

# Evaluate the model
print("ROC AUC Score:", roc_auc_score(y_test, predictions.flatten()))

# Adjust threshold and compute binary outcomes
threshold = 0.01
predicted_classes = (predictions > threshold).astype(int)

# Calculate and display the confusion matrix
cm = confusion_matrix(y_test, predicted_classes)
TN, FP, FN, TP = cm.ravel()

# Calculate TPR and TNR
TPR = TP / (TP + FN)  # True Positive Rate
TNR = TN / (TN + FP)  # True Negative Rate

# Calculate the G-Mean
G_mean = np.sqrt(TPR * TNR)

# Display TPR, TNR, and G-Mean
print("True Positive Rate (TPR):", TPR)
print("True Negative Rate (TNR):", TNR)
print("Geometric Mean (G-Mean):", G_mean)

# Detailed classification report
print(classification_report(y_test, predicted_classes))

Epoch 1/50


C:\Users\temir\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:86: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


4359/4359 ━━━━━━━━━━━━━━━━━━━━ 7s 1ms/step - accuracy: 0.9881 - loss: 0.0825 - val_accuracy: 0.9905 - val_loss: 0.0479
Epoch 2/50
4359/4359 ━━━━━━━━━━━━━━━━━━━━ 10s 1ms/step - accuracy: 0.9899 - loss: 0.0533 - val_accuracy: 0.9905 - val_loss: 0.0456
Epoch 3/50
4359/4359 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step - accuracy: 0.9900 - loss: 0.0499 - val_accuracy: 0.9905 - val_loss: 0.0462
Epoch 4/50
4359/4359 ━━━━━━━━━━━━━━━━━━━━ 6s 1ms/step - accuracy: 0.9899 - loss: 0.0477 - val_accuracy: 0.9905 - val_loss: 0.0456
Epoch 5/50
4359/4359 ━━━━━━━━━━━━━━━━━━━━ 6s 1ms/step - accuracy: 0.9896 - loss: 0.0483 - val_accuracy: 0.9905 - val_loss: 0.0460
Epoch 6/50
4359/4359 ━━━━━━━━━━━━━━━━━━━━ 4s 973us/step - accuracy: 0.9900 - loss: 0.0458 - val_accuracy: 0.9905 - val_loss: 0.0447
Epoch 7/50
4359/4359 ━━━━━━━━━━━━━━━━━━━━ 5s 984us/step - accuracy: 0.9899 - loss: 0.0467 - val_accuracy: 0.9905 - val_loss: 0.0443
Epoch 8/50
4359/4359 ━━━━━━━━━━━━━━━━━━━━ 7s 1ms/step - accuracy: 0.9899 - loss: 0.0458 - val_ac

1211/1211 ━━━━━━━━━━━━━━━━━━━━ 1s 808us/step
ROC AUC Score: 0.8628613927832509
True Positive Rate (TPR): 0.7803617571059431
True Negative Rate (TNR): 0.7979558313560868
Geometric Mean (G-Mean): 0.7891097608381039
              precision    recall  f1-score   support

           0       1.00      0.80      0.89     38353
           1       0.04      0.78      0.07       387

    accuracy                           0.80     38740
   macro avg       0.52      0.79      0.48     38740
weighted avg       0.99      0.80      0.88     38740



In [56]:
#RUS to majority class to be equal to 4 * minority class
ratio = 4
# Here, we match the number of instances in the minority class
majority_undersampled = majority.sample(n=ratio*len(minority), random_state=101)  # `random_state` ensures reproducibility
# Concatenate the minority class with the undersampled majority class
df_RUS_2 = pd.concat([majority_undersampled, minority])
# Shuffle the dataframe to prevent the model from learning any order
df_RUS_2 = df_RUS_2.sample(frac=1, random_state=42).reset_index(drop=True)
# Verify the number of instances in each class
print(df_RUS_2['fraud'].value_counts())
df_RUS_2.shape

fraud
0    7748
1    1937
Name: count, dtype: int64


(9685, 153)

In [38]:
X = df_RUS_2.drop(['fraud', 'rndrng_npi'], axis=1)
y = df_RUS_2['fraud']

# Splitting data into training and testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Normalizing the data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Building the neural network model
model = Sequential([
    Dense(32, input_dim=X_train_scaled.shape[1], activation='relu'),
    Dropout(0.5),
    Dense(32, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(X_train_scaled, y_train, epochs=50, batch_size=32, validation_split=0.1)

# Predict probabilities for the test set
predictions = model.predict(X_test_scaled)

# Evaluate the model
print("ROC AUC Score:", roc_auc_score(y_test, predictions.flatten()))

# Adjust threshold and compute binary outcomes
threshold = 0.2
predicted_classes = (predictions > threshold).astype(int)

# Calculate and display the confusion matrix
cm = confusion_matrix(y_test, predicted_classes)
TN, FP, FN, TP = cm.ravel()

# Calculate TPR and TNR
TPR = TP / (TP + FN)  # True Positive Rate: TP / (TP + FN)
TNR = TN / (TN + FP)  # True Negative Rate: TN / (TN + FP)

# Display TPR and TNR
print("True Positive Rate (TPR):", TPR)
print("True Negative Rate (TNR):", TNR)

# Detailed classification report
print(classification_report(y_test, predicted_classes))

Epoch 1/50


C:\Users\temir\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:86: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


218/218 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7908 - loss: 0.6436 - val_accuracy: 0.8129 - val_loss: 0.4945
Epoch 2/50
218/218 ━━━━━━━━━━━━━━━━━━━━ 0s 889us/step - accuracy: 0.7958 - loss: 0.5129 - val_accuracy: 0.8129 - val_loss: 0.4836
Epoch 3/50
218/218 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7978 - loss: 0.5138 - val_accuracy: 0.8129 - val_loss: 0.4833
Epoch 4/50
218/218 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8047 - loss: 0.4979 - val_accuracy: 0.8129 - val_loss: 0.4839
Epoch 5/50
218/218 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7955 - loss: 0.5165 - val_accuracy: 0.8129 - val_loss: 0.4833
Epoch 6/50
218/218 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7935 - loss: 0.5177 - val_accuracy: 0.8129 - val_loss: 0.4829
Epoch 7/50
218/218 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7990 - loss: 0.5077 - val_accuracy: 0.8129 - val_loss: 0.4833
Epoch 8/50
218/218 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7919 - loss: 0.5164 - val_accuracy: 0.8129 - va

61/61 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
ROC AUC Score: 0.5
True Positive Rate (TPR): 0.0
True Negative Rate (TNR): 1.0
              precision    recall  f1-score   support

           0       0.80      1.00      0.89      1550
           1       0.00      0.00      0.00       387

    accuracy                           0.80      1937
   macro avg       0.40      0.50      0.44      1937
weighted avg       0.64      0.80      0.71      1937



C:\Users\temir\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\temir\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\temir\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [68]:
#RUS to majority class to be equal to 1.5 * minority class
ratio = 1.5
# Here, we match the number of instances in the minority class
majority_undersampled = majority.sample(n=int(ratio*len(minority)), random_state=1011)  # `random_state` ensures reproducibility
# Concatenate the minority class with the undersampled majority class
df_RUS_3 = pd.concat([majority_undersampled, minority])
# Shuffle the dataframe to prevent the model from learning any order
df_RUS_3 = df_RUS_3.sample(frac=1, random_state=42).reset_index(drop=True)
# Verify the number of instances in each class
print(df_RUS_3['fraud'].value_counts())
df_RUS_3.shape

fraud
0    2905
1    1937
Name: count, dtype: int64


(4842, 153)

In [69]:
X = df_RUS_3.drop(['fraud', 'rndrng_npi'], axis=1)
y = df_RUS_3['fraud']

# Splitting data into training and testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Normalizing the data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Building the neural network model
model = Sequential([
    Dense(32, input_dim=X_train_scaled.shape[1], activation='relu'),
    Dropout(0.5),
    Dense(32, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(X_train_scaled, y_train, epochs=50, batch_size=32, validation_split=0.1)

# Predict probabilities for the test set
predictions = model.predict(X_test_scaled)

# Evaluate the model
print("ROC AUC Score:", roc_auc_score(y_test, predictions.flatten()))

# Adjust threshold and compute binary outcomes
threshold = 0.4
predicted_classes = (predictions > threshold).astype(int)

# Calculate and display the confusion matrix
cm = confusion_matrix(y_test, predicted_classes)
TN, FP, FN, TP = cm.ravel()

# Calculate TPR and TNR
TPR = TP / (TP + FN)  # True Positive Rate: TP / (TP + FN)
TNR = TN / (TN + FP)  # True Negative Rate: TN / (TN + FP)

# Display TPR and TNR
print("True Positive Rate (TPR):", TPR)
print("True Negative Rate (TNR):", TNR)

# Detailed classification report
print(classification_report(y_test, predicted_classes))

Epoch 1/50


C:\Users\temir\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:86: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


109/109 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.5092 - loss: 0.8698 - val_accuracy: 0.7036 - val_loss: 0.5933
Epoch 2/50
109/109 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6435 - loss: 0.6559 - val_accuracy: 0.7268 - val_loss: 0.5508
Epoch 3/50
109/109 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6622 - loss: 0.6453 - val_accuracy: 0.7552 - val_loss: 0.5244
Epoch 4/50
109/109 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7080 - loss: 0.6009 - val_accuracy: 0.7526 - val_loss: 0.5069
Epoch 5/50
109/109 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7182 - loss: 0.5785 - val_accuracy: 0.7655 - val_loss: 0.4930
Epoch 6/50
109/109 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7160 - loss: 0.5537 - val_accuracy: 0.7629 - val_loss: 0.4836
Epoch 7/50
109/109 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7564 - loss: 0.5324 - val_accuracy: 0.7629 - val_loss: 0.4806
Epoch 8/50
109/109 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7563 - loss: 0.5283 - val_accuracy: 0.7706 - val_

31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
ROC AUC Score: 0.8651498482885889
True Positive Rate (TPR): 0.7835051546391752
True Negative Rate (TNR): 0.7986230636833046
              precision    recall  f1-score   support

           0       0.85      0.80      0.82       581
           1       0.72      0.78      0.75       388

    accuracy                           0.79       969
   macro avg       0.78      0.79      0.79       969
weighted avg       0.80      0.79      0.79       969



In [72]:
#RUS to majority class to be equal to 1 * minority class
ratio = 1
# Here, we match the number of instances in the minority class
majority_undersampled = majority.sample(n=int(ratio*len(minority)), random_state=77)  # `random_state` ensures reproducibility
# Concatenate the minority class with the undersampled majority class
df_RUS_4 = pd.concat([majority_undersampled, minority])
# Shuffle the dataframe to prevent the model from learning any order
df_RUS_4 = df_RUS_4.sample(frac=1, random_state=42).reset_index(drop=True)
# Verify the number of instances in each class
print(df_RUS_4['fraud'].value_counts())
df_RUS_4.shape

fraud
1    1937
0    1937
Name: count, dtype: int64


(3874, 153)

In [73]:
X = df_RUS_4.drop(['fraud', 'rndrng_npi'], axis=1)
y = df_RUS_4['fraud']

# Splitting data into training and testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Normalizing the data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Building the neural network model
model = Sequential([
    Dense(32, input_dim=X_train_scaled.shape[1], activation='relu'),
    Dropout(0.5),
    Dense(32, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(X_train_scaled, y_train, epochs=50, batch_size=32, validation_split=0.1)

# Predict probabilities for the test set
predictions = model.predict(X_test_scaled)

# Evaluate the model
print("ROC AUC Score:", roc_auc_score(y_test, predictions.flatten()))

# Adjust threshold and compute binary outcomes
threshold = 0.5
predicted_classes = (predictions > threshold).astype(int)

# Calculate and display the confusion matrix
cm = confusion_matrix(y_test, predicted_classes)
TN, FP, FN, TP = cm.ravel()

# Calculate TPR and TNR
TPR = TP / (TP + FN)  # True Positive Rate: TP / (TP + FN)
TNR = TN / (TN + FP)  # True Negative Rate: TN / (TN + FP)

# Display TPR and TNR
print("True Positive Rate (TPR):", TPR)
print("True Negative Rate (TNR):", TNR)

# Detailed classification report
print(classification_report(y_test, predicted_classes))

Epoch 1/50


C:\Users\temir\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:86: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


88/88 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.5395 - loss: 0.8016 - val_accuracy: 0.6839 - val_loss: 0.6428
Epoch 2/50
88/88 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6191 - loss: 0.7009 - val_accuracy: 0.7484 - val_loss: 0.6117
Epoch 3/50
88/88 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6428 - loss: 0.6526 - val_accuracy: 0.7548 - val_loss: 0.5839
Epoch 4/50
88/88 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6743 - loss: 0.6244 - val_accuracy: 0.7484 - val_loss: 0.5697
Epoch 5/50
88/88 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7120 - loss: 0.6024 - val_accuracy: 0.7613 - val_loss: 0.5656
Epoch 6/50
88/88 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7172 - loss: 0.5889 - val_accuracy: 0.7645 - val_loss: 0.5564
Epoch 7/50
88/88 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7369 - loss: 0.5701 - val_accuracy: 0.7645 - val_loss: 0.5513
Epoch 8/50
88/88 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7352 - loss: 0.5820 - val_accuracy: 0.7774 - val_loss: 0.5426
Epo

25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
ROC AUC Score: 0.8715669037534299
True Positive Rate (TPR): 0.8372093023255814
True Negative Rate (TNR): 0.7912371134020618
              precision    recall  f1-score   support

           0       0.83      0.79      0.81       388
           1       0.80      0.84      0.82       387

    accuracy                           0.81       775
   macro avg       0.81      0.81      0.81       775
weighted avg       0.81      0.81      0.81       775



In [75]:
#RUS to majority class to be equal to 1.5 * minority class
ratio = 2/3
# Here, we match the number of instances in the minority class
majority_undersampled = majority.sample(n=int(ratio*len(minority)), random_state=707)  # `random_state` ensures reproducibility
# Concatenate the minority class with the undersampled majority class
df_RUS_5 = pd.concat([majority_undersampled, minority])
# Shuffle the dataframe to prevent the model from learning any order
df_RUS_5 = df_RUS_5.sample(frac=1, random_state=42).reset_index(drop=True)
# Verify the number of instances in each class
print(df_RUS_5['fraud'].value_counts())
df_RUS_5.shape

fraud
1    1937
0    1291
Name: count, dtype: int64


(3228, 153)

In [81]:
X = df_RUS_5.drop(['fraud', 'rndrng_npi'], axis=1)
y = df_RUS_5['fraud']

# Splitting data into training and testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Normalizing the data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Building the neural network model
model = Sequential([
    Dense(32, input_dim=X_train_scaled.shape[1], activation='relu'),
    Dropout(0.5),
    Dense(32, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(X_train_scaled, y_train, epochs=50, batch_size=32, validation_split=0.1)

# Predict probabilities for the test set
predictions = model.predict(X_test_scaled)

# Evaluate the model
print("ROC AUC Score:", roc_auc_score(y_test, predictions.flatten()))

# Adjust threshold and compute binary outcomes
threshold = 0.59
predicted_classes = (predictions > threshold).astype(int)

# Calculate and display the confusion matrix
cm = confusion_matrix(y_test, predicted_classes)
TN, FP, FN, TP = cm.ravel()

# Calculate TPR and TNR
TPR = TP / (TP + FN)  # True Positive Rate
TNR = TN / (TN + FP)  # True Negative Rate

# Calculate the G-Mean
G_mean = np.sqrt(TPR * TNR)

# Display TPR, TNR, and G-Mean
print("True Positive Rate (TPR):", TPR)
print("True Negative Rate (TNR):", TNR)
print("Geometric Mean (G-Mean):", G_mean)

# Detailed classification report
print(classification_report(y_test, predicted_classes))

Epoch 1/50


C:\Users\temir\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:86: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


73/73 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.5615 - loss: 0.8321 - val_accuracy: 0.6718 - val_loss: 0.6373
Epoch 2/50
73/73 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5970 - loss: 0.7097 - val_accuracy: 0.7066 - val_loss: 0.6061
Epoch 3/50
73/73 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6328 - loss: 0.6483 - val_accuracy: 0.7375 - val_loss: 0.5862
Epoch 4/50
73/73 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6763 - loss: 0.6356 - val_accuracy: 0.7568 - val_loss: 0.5647
Epoch 5/50
73/73 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6849 - loss: 0.6216 - val_accuracy: 0.7568 - val_loss: 0.5466
Epoch 6/50
73/73 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6985 - loss: 0.5885 - val_accuracy: 0.7722 - val_loss: 0.5371
Epoch 7/50
73/73 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7488 - loss: 0.5412 - val_accuracy: 0.7799 - val_loss: 0.5275
Epoch 8/50
73/73 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7395 - loss: 0.5577 - val_accuracy: 0.7761 - val_loss: 0.5250
Epo

21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
ROC AUC Score: 0.8527831055702071
True Positive Rate (TPR): 0.8247422680412371
True Negative Rate (TNR): 0.7403100775193798
Geometric Mean (G-Mean): 0.7813865959863385
              precision    recall  f1-score   support

           0       0.74      0.74      0.74       258
           1       0.83      0.82      0.83       388

    accuracy                           0.79       646
   macro avg       0.78      0.78      0.78       646
weighted avg       0.79      0.79      0.79       646



In [2]:
df_final = pd.read_csv("data_set_result_aggregated_by_npi.csv")
df_final.fillna(0, inplace=True)
df_final_1 = pd.read_csv("data_set_result_yearly.csv")
df_final_1.fillna(0, inplace=True)

In [6]:
X = df_final.drop(['fraud', 'rndrng_npi'], axis=1)
y = df_final['fraud']
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Normalize the training and validation data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)



# Build the neural network model
model = Sequential([
    Dense(32, input_dim=X_train_scaled.shape[1], activation='relu'),
    Dropout(0.5),
    Dense(32, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(X_train_scaled, y_train, epochs=10, batch_size=32, validation_data=(X_val_scaled, y_val))


C:\Users\temir\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:86: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
37372/37372 ━━━━━━━━━━━━━━━━━━━━ 27s 680us/step - accuracy: 0.9973 - loss: 0.0173 - val_accuracy: 0.9987 - val_loss: 0.0094
Epoch 2/10
37372/37372 ━━━━━━━━━━━━━━━━━━━━ 26s 681us/step - accuracy: 0.9987 - loss: 0.0102 - val_accuracy: 0.9987 - val_loss: 0.0093
Epoch 3/10
37372/37372 ━━━━━━━━━━━━━━━━━━━━ 25s 675us/step - accuracy: 0.9987 - loss: 0.0102 - val_accuracy: 0.9987 - val_loss: 0.0091
Epoch 4/10
37372/37372 ━━━━━━━━━━━━━━━━━━━━ 25s 675us/step - accuracy: 0.9987 - loss: 0.0101 - val_accuracy: 0.9987 - val_loss: 0.0093
Epoch 5/10
37372/37372 ━━━━━━━━━━━━━━━━━━━━ 25s 676us/step - accuracy: 0.9986 - loss: 0.0105 - val_accuracy: 0.9987 - val_loss: 0.0092
Epoch 6/10
37372/37372 ━━━━━━━━━━━━━━━━━━━━ 26s 685us/step - accuracy: 0.9987 - loss: 0.0103 - val_accuracy: 0.9987 - val_loss: 0.0093
Epoch 7/10
37372/37372 ━━━━━━━━━━━━━━━━━━━━ 25s 668us/step - accuracy: 0.9987 - loss: 0.0102 - val_accuracy: 0.9987 - val_loss: 0.0093
Epoch 8/10
37372/37372 ━━━━━━━━━━━━━━━━━━━━ 25s 680us/s

In [33]:
# Prepare the second dataset for testing

X_1 = df_final_1.drop(['fraud', 'rndrng_npi'], axis=1)
y_1 = df_final_1['fraud']
X_1_scaled = scaler.transform(X_1)  # Use the same scaler without fitting it again

predictions_1 = model.predict(X_1_scaled)



270811/270811 ━━━━━━━━━━━━━━━━━━━━ 108s 400us/step


In [34]:
len(X_1)
len(y_1)
s= 0 
for i in y_1:
    s = s+i
s

8665935

8665935

8118

In [42]:
# Adjust threshold and compute binary outcomes
threshold = 0.0027  # Adjust the threshold as needed
predicted_classes_1 = (predictions_1 > threshold).astype(int)

# Calculate confusion matrix
cm = confusion_matrix(y_1, predicted_classes_1)
TN, FP, FN, TP = cm.ravel()

# Calculate TPR (True Positive Rate) and TNR (True Negative Rate)
TPR = TP / (TP + FN) if (TP + FN) != 0 else 0
TNR = TN / (TN + FP) if (TN + FP) != 0 else 0

# Calculate Geometric Mean (G-Mean)
G_Mean = np.sqrt(TPR * TNR)

# Print results
print("ROC AUC Score:", roc_auc_score(y_1, predictions_1.flatten()))
print("True Positive Rate (TPR):", TPR)
print("True Negative Rate (TNR):", TNR)
print("Geometric Mean (G-Mean):", G_Mean)
print(classification_report(y_1, predicted_classes_1))

ROC AUC Score: 0.7627935523863829
True Positive Rate (TPR): 0.7099039172209904
True Negative Rate (TNR): 0.6913404383576137
Geometric Mean (G-Mean): 0.7005606935329348
              precision    recall  f1-score   support

           0       1.00      0.69      0.82   8657817
           1       0.00      0.71      0.00      8118

    accuracy                           0.69   8665935
   macro avg       0.50      0.70      0.41   8665935
weighted avg       1.00      0.69      0.82   8665935

